In [1]:
import quantumpropagator as qp
import matplotlib.pyplot as plt
import matplotlib
from ipywidgets import interact,fixed #, interactive, fixed, interact_manual
import ipywidgets as widgets

import os
import sys
import glob
import pandas as pd
import numpy as np

%matplotlib ipympl

plt.rcParams.update({'font.size': 6})

In [2]:
%matplotlib ipympl
fol = '/home/alessio/w-August-Run/'
outputFolder = '/home/alessio/w-August-Run/OUTPUTS'
subfolders2 = sorted([dir for dir in os.listdir(fol) if os.path.isdir(os.path.join(fol,dir))])
subfolders =  [ dir for dir in subfolders2 if dir not in ['HTML','csv','OUTPUTS','FRANCOISE','.ipynb_checkpoints'] ]
print(''.join(['{} -> {}\n'.format(a,b) for a,b in enumerate(subfolders)]))

0 -> a-UV-Pump-Probe-2687_0000
1 -> a-UV-Pump-Probe-2791_0000
2 -> a-UV-Pump-Probe-2894_0000
3 -> a-UV-Pump-Probe-2997_0000
4 -> a-UV-Pump-Probe-3101_0000
5 -> b-UV-0.22_0000
6 -> d-UVz-Pump-Probe-1860_0000
7 -> d-UVz-Pump-Probe-2248_0000
8 -> d-UVz-Pump-Probe-2401_0000
9 -> d-UVz-Pump-Probe-2563_0000
10 -> d-UVz-Pump-Probe-3059_0000
11 -> m-IR-Polarized-INV_0000
12 -> m-IR-Polarized-LON_0000
13 -> m-IR-Polarized-NOR_0000
14 -> n-target-2-3-only-diffCAP_0000
15 -> o-UV-Pump-Probe-0000_0000
16 -> o-UV-Pump-Probe-0001_0000
17 -> o-UV-Pump-Probe-0002_0000
18 -> o-UV-Pump-Probe-0003_0000
19 -> o-UV-Pump-Probe-0004_0000
20 -> o-UV-Pump-Probe-0005_0000
21 -> o-UV-Pump-Probe-0006_0000
22 -> o-UV-Pump-Probe-0007_0000
23 -> o-UV-Pump-Probe-0008_0000
24 -> o-UV-Pump-Probe-0009_0000
25 -> o-UV-Pump-Probe-0010_0000
26 -> o-UV-Pump-Probe-0011_0000
27 -> o-UV-Pump-Probe-0012_0000
28 -> o-UV-Pump-Probe-0013_0000
29 -> o-UV-Pump-Probe-0014_0000
30 -> o-UV-Pump-Probe-0015_0000
31 -> o-UV-Pump-Probe-001

In [3]:
def process_folder(this_folder,fol,outputFolder):
    '''
    this function will create a correct set of files to give to Francoise
    '''
    project_folder = os.path.join(fol,this_folder)
    project_output = os.path.join(outputFolder,this_folder)
    output_norm = os.path.join(project_folder, 'output')
    output_popu = os.path.join(project_folder, 'outputPopul')
    output_abso = os.path.join(project_folder, 'Output_Abs')
    output_regi = os.path.join(project_folder, 'Output_Regions')
    output_dipo = os.path.join(project_folder, 'Output_Dipole')
    
    df_norm2 = pd.read_csv(output_norm, delim_whitespace=True, index_col=0, names=['counter', 'steps', 'fs','Norm deviation','Kinetic','Potential','Total','Total Deviation','Xpulse','Ypulse','Zpulse','AbZino'])
    df_popu2 = pd.read_csv(output_popu, delim_whitespace=True, names=['fs_1', 'S0', 'S1','S2','S3','S4','S5','S6','S7'])
    df_abso2 = pd.read_csv(output_abso, delim_whitespace=True, names=['Time AU', 'Abs Tot', 'Abs S0', 'Abs S1','Abs S2','Abs S3','Abs S4','Abs S5','Abs S6','Abs S7'])
    df_regi2 = pd.read_csv(output_regi, delim_whitespace=True, names=['Non-decided','Reactants','Products'])
    
    # I need to cut down to different file size <- this will become obsolete
    dfs = [df_norm2,df_popu2,df_abso2,df_regi2]
    lengths = min([x.shape[0] for x in dfs])
    df_norm, df_popu, df_abso, df_regi = [ x.drop(x.index[lengths:]) for x in dfs ]

    df_all = pd.concat([df_norm, df_popu, df_abso, df_regi], axis = 1)

    # time step in AU
    AU_dt = df_abso['Time AU'].iloc[1]-df_abso['Time AU'].iloc[0]
    
    # create the absorbed columns
    for statZ in ['Tot','S0','S1','S2','S3','S4','S5','S6','S7']:
        df_all['P(t) {}'.format(statZ)] = -(np.cumsum(df_all['Abs {}'.format(statZ)])*AU_dt)
    
    # change sign of norm
    df_all['Norm deviation'] = -df_all['Norm deviation']
    
    # clean the paper
    final_df = df_all.drop(['Abs Tot', 'Abs S0', 'Abs S1','Abs S2','Abs S3','Abs S4','Abs S5','Abs S6','Abs S7','fs_1','AbZino'],axis=1)
    
    csv_name = '{}.csv'.format(project_output)
    final_df.to_csv(csv_name)
    #print(list(final_df.columns) )
    

process_folder(subfolders[38],fol,outputFolder)
process_folder(subfolders[40],fol,outputFolder)

# process_folder(subfolders[26],fol,outputFolder)
# process_folder(subfolders[27],fol,outputFolder)
# process_folder(subfolders[28],fol,outputFolder)
# process_folder(subfolders[29],fol,outputFolder)
# process_folder(subfolders[30],fol,outputFolder)
    
process_folder(subfolders[0],fol,outputFolder)
process_folder(subfolders[1],fol,outputFolder)
process_folder(subfolders[2],fol,outputFolder)
process_folder(subfolders[3],fol,outputFolder)
process_folder(subfolders[46],fol,outputFolder)

In [4]:
kind = 'r_c_1.4-non-overlapping'
bonds_folder = '/home/alessio/densities/From_pinot/calculationZ_high_def/{}'.format(kind)

def create_bonds_files(bonds_folder_first, outputFolder):
    for first_folder in ['ACTIVE','ALL']:
        bonds_folder = os.path.join(bonds_folder_first,first_folder)
        for next_folder in ['no_s0', 'with_s0']:
            sub3 = os.path.join(bonds_folder,next_folder)
            subfolders2 = sorted([dir for dir in os.listdir(sub3) if os.path.isdir(os.path.join(sub3,dir))])
            for proj in subfolders2:
                full_path = os.path.join(sub3,proj)
                files_dat = sorted([ os.path.join(full_path,x) for x in os.listdir(full_path) if x[-4:] == '.dat' and x[0] == 'l' ])
                data = [ np.loadtxt(i) for i in files_dat ]
                dic = {}
                dic['Time'] = data[0][:,0]  # first column of first file, the time
                labels = ['Old bond','New Bond']
                for i, this_one in enumerate(data):
                    dic[labels[i]] = this_one[:,1] # second column, the value
                df = pd.DataFrame(dic)
                file_final_name = '{}_BONDS_{}_{}_{}.csv'.format(proj,kind,first_folder,next_folder)
                print('file {} written'.format(file_final_name))
                df.to_csv(os.path.join(outputFolder,file_final_name))
    return(df)
    

a = create_bonds_files(bonds_folder, outputFolder)


file b-UV-0.22_0000_BONDS_r_c_1.4-non-overlapping_ACTIVE_no_s0.csv written
file m-IR-Polarized-LON_0000_BONDS_r_c_1.4-non-overlapping_ACTIVE_no_s0.csv written
file m-IR-Polarized-NOR_0000_BONDS_r_c_1.4-non-overlapping_ACTIVE_no_s0.csv written
file u-target-2-3-only_0000_BONDS_r_c_1.4-non-overlapping_ACTIVE_no_s0.csv written
file b-UV-0.22_0000_BONDS_r_c_1.4-non-overlapping_ACTIVE_with_s0.csv written
file m-IR-Polarized-LON_0000_BONDS_r_c_1.4-non-overlapping_ACTIVE_with_s0.csv written
file m-IR-Polarized-NOR_0000_BONDS_r_c_1.4-non-overlapping_ACTIVE_with_s0.csv written
file u-target-2-3-only_0000_BONDS_r_c_1.4-non-overlapping_ACTIVE_with_s0.csv written
file b-UV-0.22_0000_BONDS_r_c_1.4-non-overlapping_ALL_no_s0.csv written
file m-IR-Polarized-LON_0000_BONDS_r_c_1.4-non-overlapping_ALL_no_s0.csv written
file m-IR-Polarized-NOR_0000_BONDS_r_c_1.4-non-overlapping_ALL_no_s0.csv written
file u-target-2-3-only_0000_BONDS_r_c_1.4-non-overlapping_ALL_no_s0.csv written
file b-UV-0.22_0000_BONDS_

# Product/Reactants part 4

In [5]:
names = ['b-UV-0.22_0000.csv', 'm-IR-Polarized-LON_0000.csv', 'm-IR-Polarized-INV_0000.csv', 'm-IR-Polarized-NOR_0000.csv', 'z-from1_0000.csv', 'u-target-2-3-only_0000.csv', 'u-target-2-3-only_mask_0000.csv']
main_papers = [ pd.read_csv(os.path.join(outputFolder,x)) for x in names ]

masknames = {'b-UV-0.22_0000.csv' : 'UV', 'm-IR-Polarized-LON_0000.csv' : 'IR_LON', 'm-IR-Polarized-INV_0000.csv' : 'IR_INV', 'm-IR-Polarized-NOR_0000.csv' : 'IR_NOR', 'z-from1_0000.csv' : 'FC', 'u-target-2-3-only_0000.csv' : '2-3 IR', 'u-target-2-3-only_mask_0000.csv' : '2-3 IR Mask'}

In [6]:
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'mediumpurple']
plt.close('all')

def get_product_reactants(dfs,names,output_name):
    '''little function to mimic calculate/ratio3'''
    fig, [[ax0, ax2],[ax4,ax6]] = plt.subplots(2,2,figsize=(16,8))
    
    for i, df in enumerate(dfs):
        this_lab = masknames[names[i]]
        ax0.plot(df['fs'], df['Reactants'], label=this_lab, ls='--', color=colors[i])
        ax0.plot(df['fs'], df['Products'], label=this_lab ,          color=colors[i])
        ax0.set_title('JUST REGIONS')
        
        ax2.plot(df['fs'], df['P(t) S0'], label=this_lab, color=colors[i])
        ax2.set_title('JUST ABSORBED S0')
    
        ax4.plot(df['fs'], df['Products'] + df['P(t) S0'], label=this_lab, color=colors[i])
        ax4.set_title('SUM OF ABOVE')
        
        if this_lab == '2-3 IR':
            ax6.plot(df['fs'], (df['Products'] + df['P(t) Tot']), label=this_lab, color=colors[i])
            ax6.set_title('SUM with all ABSORBED STATES')
        else:
            ax6.plot(df['fs'], df['Products'] + df['P(t) Tot'], label=this_lab, color=colors[i])
            ax6.set_title('SUM with all ABSORBED STATES')
                
        #ax7.plot(df['fs'], df['Products'] + df['P(t) S6'], label=this_lab, color=colors[i])
        
        
        number_line_after_pulse = 400
        fs_after_pulse = df['fs'].iloc[number_line_after_pulse]
    
        all_state_labels = ['S0','S1','S2','S3','S4','S5','S6','S7']
        all_after_pulse = np.empty(len(all_state_labels))
        for ii, this_state_label in enumerate(all_state_labels):
            all_after_pulse[ii] = df[this_state_label].iloc[number_line_after_pulse]
            # print(df[this_state_label].iloc[number_line_after_pulse])
        
    fig.canvas.layout.height = '800px'
    
        #fig.canvas.layout.width = '500px'

    ax0.legend()
    ax2.legend()
    ax4.legend()
    ax6.legend()

    fig.tight_layout()
    fig.savefig(output_name)


output_name = '/home/alessio/u-3dPropagation_manuscript/Report/images/prodreact.png'
get_product_reactants(main_papers,names,output_name)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
print(list(enumerate(names)))

[(0, 'b-UV-0.22_0000.csv'), (1, 'm-IR-Polarized-LON_0000.csv'), (2, 'm-IR-Polarized-INV_0000.csv'), (3, 'm-IR-Polarized-NOR_0000.csv'), (4, 'z-from1_0000.csv'), (5, 'u-target-2-3-only_0000.csv'), (6, 'u-target-2-3-only_mask_0000.csv')]


In [10]:
df = main_papers[6]

threshold = 0.0000000001 # threshold at which the pulse is considered finished.
pulse_norm = np.linalg.norm(np.stack((df['Xpulse'],df['Ypulse'],df['Zpulse']),axis=1),axis=1)
number_line_after_pulse_array = np.where(pulse_norm > threshold)[0]

# FC is empty
if number_line_after_pulse_array.size == 0:
    number_line_after_pulse = 0
else:
    number_line_after_pulse = number_line_after_pulse_array[-1]

fs_after_pulse = df['fs'].iloc[number_line_after_pulse]
fs_after_pulse

58.808856085411605